<a id="1"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Train </p>
### This notebook is a follow-up of [🌟🐟Detection using Keras-RetinaNet-Train Notebook](https://www.kaggle.com/mahipalsingh/detection-using-keras-retinanet-train)

<a id="2"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Import Modules </p>

In [1]:
import pandas as pd
# show images inline
%matplotlib inline
import keras
import tensorflow
# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import tensorflow as tf

# Create 'RetinaNet' dir for zip extraction
%mkdir RetinaNet

#%cd RetinaNet/
from distutils.dir_util import copy_tree

f='/kaggle/input/offlineretinanet/'
t='/kaggle/working/RetinaNet/'
copy_tree(f,t)

<a id="3"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Install RetinaNet Offline  </p>

In [1]:
#Clone Git Repository
#!git clone https://github.com/fizyr/keras-retinanet.git
#%cd keras-retinanet/
%cd RetinaNet/
!python setup.py build_ext --inplace

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet import models

!pip install /kaggle/input/creating-whl-file-retinanet/whlfiles/keras_resnet-0.2.0-py2.py3-none-any.whl
import keras
import keras_resnet

<a id="4"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Load Model  </p>

In [1]:
model_path = '/kaggle/input/v2models/bestRetinanet.h5'
#print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')  ## Use backbone as resnet50
model = models.convert_model(model)

# load label to names mapping for visualization purposes
#labels_to_names = pd.read_csv('/kaggle/input/detection-using-keras-retinanet-train/keras-retinanet/classes.csv',header=None).T.loc[0].to_dict()

In [1]:
#Annotation file for visualizing result
df_extrain_main=pd.read_csv("/kaggle/input/v2models/annotation.csv",names=['image_path','x','y','width','height',"class"])
df_extrain_main

<a id="5"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Visualizing Results  </p>

In [1]:
THRES_SCORE = 0.5  # Set Score Threshold Value

def df_plot_orinal(drawOG,img_path,df):
    df=df[df['image_path']==img_path]
    for i,r in df.iterrows():
        cv2.rectangle(drawOG, (r['x'], r['y']), (r['width'], r['height']), (255,0,0),2)
    

def img_inference(img_path):
  image = read_image_bgr(img_path)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
  drawOG = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  df_plot_orinal(drawOG,img_path,df_extrain_main)
  # correct for image scale
  boxes /= scale
  #print(boxes)
  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      #print(score)
      if score < THRES_SCORE:
          continue
      #print(score)
      color = label_color(label)
      b = box.astype(int)
      draw_box(draw, b, color=color)
      #caption = "{} {:.3f}%".format(labels_to_names[label], score*100)
    
  fig = plt.figure(figsize=(20, 20))
  ax1=fig.add_subplot(1, 2, 1)
  plt.imshow(draw)
  ax2=fig.add_subplot(1, 2, 2)
  plt.imshow(drawOG)

  ax1.title.set_text('Predicted')
  ax2.title.set_text('Actual')
  plt.show()

In [1]:
data=df_extrain_main.sample(n=10)  #Predict on Random 10 Image
for i,r in data.iterrows():
    img_inference(r['image_path'])

<a id="5"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Make Submission  </p>

In [1]:
%cd /kaggle/working/

In [1]:
import greatbarrierreef

env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()

In [1]:
submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image, sample_prediction_df) in iter_test:
  #image = read_image_bgr(Image.fromarray(image))
  image = preprocess_image(image[:, :, ::-1])
  image, scale = resize_image(image)

  bboxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  #bboxes_, scores_, labels_=bboxes[0], scores[0], labels[0]
  # correct for image scale
  bboxes /= scale
  predictions = []
  for box, score, label in zip(bboxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      if score < THRES_SCORE:
          continue
      x_min = int(box[0])  
      y_min = int(box[1])
      x_max = int(box[2])
      y_max = int(box[3])

      bbox_width = x_max - x_min
      bbox_height = y_max - y_min
      predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))

  prediction_str = ' '.join(predictions)
  sample_prediction_df['annotations'] = prediction_str
  env.predict(sample_prediction_df)

print('Prediction:', sample_prediction_df)

In [1]:
sub_df = pd.read_csv('submission.csv')
sub_df.head()

<p style="font-size:220%;text-align:center"> If you find this notebook interesting, please do upvote :) </p>

# <p style="text-align:center"> <img src="https://media.giphy.com/media/3oEdva9BUHPIs2SkGk/giphy.gif"> </p>

<p style="font-size:130%;text-align:center"> Still lot of improvments is remaining!... 🛠⚙ </p>

<a id="5"></a>
# <p style="background-color:#000000;font-family:newtimeroman;color:#fff;font-size:120%;text-align:center;border-radius:20px 80px;"> Reference  </p>

## [Creating WHL file - Retinanet](https://www.kaggle.com/akhileshdkapse/creating-whl-file-retinanet)